In [1]:
pip install sentence_transformers

  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.40.2-py3-none-any.whl.metadata (137 kB)
  Using cached torch-2.3.0-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached scikit_learn-1.4.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.13.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pillow-10.3.0-cp312-cp312-win_amd64.whl.metadata (9.4 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached safetensors-0.4.3-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.12.0-py3-none-win_amd64.whl.metadata (1.1 kB)
Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
Using cached torch-2.3.0-cp312-cp312-win_amd64.whl (159.7 MB)


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import chromadb
from nltk.tokenize import sent_tokenize
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, set_seed
from sentence_transformers import SentenceTransformer

In [4]:
#client.delete_collection("question_answer")

In [5]:
client = chromadb.PersistentClient(path="./chromadb")
collection = client.get_collection("question_answer")

In [6]:
articles = pd.read_csv('../data/wiki_processed.csv')
articles = articles.reset_index()
articles.head()

,index,title,link,year,text,processed_text
0,0,Gladiator,https://en.wikipedia.org/wiki/Gladiator_(2000_...,2000,Film directed by Ridley Scott\n\n\nGladiatorTh...,Film directed by Ridley Scott GladiatorTheatri...
1,1,A Beautiful Mind,https://en.wikipedia.org/wiki/A_Beautiful_Mind...,2001,2001 film by Ron Howard\n\n\nA Beautiful MindT...,2001 film by Ron Howard A Beautiful MindTheatr...
2,2,Chicago,https://en.wikipedia.org/wiki/Chicago_(2002_film),2002,"Film by Rob Marshall\n""Chicago (film)"" redirec...",Film by Rob Marshall Chicago film redirects he...
3,3,The Lord of the Rings: The Return of the King,https://en.wikipedia.org/wiki/The_Lord_of_the_...,2003,2003 film by Peter Jackson\nThis article is ab...,2003 film by Peter Jackson This article is abo...
4,4,Million Dollar Baby,https://en.wikipedia.org/wiki/Million_Dollar_Baby,2004,2004 American sports drama film by Clint Eastw...,2004 American sports drama film by Clint Eastw...


In [7]:
articles.shape

(24, 6)

In [8]:
def add_article(article):
    sentences = sent_tokenize(article['processed_text'])
    collection.add(
        documents = sentences,
        ids = [f'{article["index"]}_{i}' for i in range(len(sentences))],
        metadatas = [{'article': article['title']}] * len(sentences)
    )

In [9]:
for _, row in tqdm(articles.iterrows()):
    add_article(row)

0it [00:00, ?it/s]

Add of existing embedding ID: 0_0
Add of existing embedding ID: 1_0
Add of existing embedding ID: 2_0
Add of existing embedding ID: 3_0
Add of existing embedding ID: 4_0
Add of existing embedding ID: 5_0
Add of existing embedding ID: 6_0
Add of existing embedding ID: 7_0
Add of existing embedding ID: 8_0
Add of existing embedding ID: 9_0
Add of existing embedding ID: 10_0
Add of existing embedding ID: 11_0
Add of existing embedding ID: 12_0
Add of existing embedding ID: 13_0
Add of existing embedding ID: 14_0
Add of existing embedding ID: 15_0
Add of existing embedding ID: 16_0
Add of existing embedding ID: 17_0
Add of existing embedding ID: 18_0
Add of existing embedding ID: 19_0
Add of existing embedding ID: 20_0
Add of existing embedding ID: 21_0
Add of existing embedding ID: 22_0
Add of existing embedding ID: 23_0
Add of existing embedding ID: 0_0
Add of existing embedding ID: 1_0
Add of existing embedding ID: 2_0
Add of existing embedding ID: 3_0
Add of existing embedding ID: 4_0


In [10]:
query_text = "bootlegging"

results = collection.query(
    query_texts = [query_text],
    n_results = 5
)

print(query_text)
print('\n -------- \n')
print('\n'.join(results['documents'][0]))
print('\n --------- \n')

bootlegging

 -------- 

2015 film directed by Tom McCarthy This article is about the 2015 film For the 1927 film see The Spotlight film SpotlightTheatrical release posterDirected byTom McCarthyWritten by Josh Singer Tom McCarthy Produced by Blye Pagon Faust Steve Golin Nicole Rocklin Michael Sugar Starring Mark Ruffalo Michael Keaton Rachel McAdams Liev Schreiber John Slattery Stanley Tucci CinematographyMasanobu TakayanagiEdited byTom McArdleMusic byHoward ShoreProductioncompanies Participant Media First Look Media Anonymous Content Rocklin Faust Productions Spotlight Film Distributed by Open Road Films United States Entertainment One Features International Release dates September 3 2015 2015 09 03 Venice November 6 2015 2015 11 06 United States Running time129 minutes 1 CountryUnited StatesLanguageEnglishBudget 20 million 2 Box office 98 7 million 3 Spotlight is a 2015 American biographical drama film directed by Tom McCarthy and written by McCarthy and Josh Singer 4 5 The film foll

In [11]:
questions=pd.read_csv('../data/QAs.csv')

In [12]:
questions

,film,question,answer
0,Gladiator,What was the worldwide gross of Gladiator?,$465.4 million
1,A Beautiful Mind,Who was consulted on the mathematical equation...,Dave Bayer
2,Chicago,Where did principal photography take place for...,"Toronto, Ontario, Canada."
3,The Lord of the Rings: The Return of the King,Who composed the score for the film The Lord o...,Howard Shore
4,Million Dollar Baby,What day was Million Dollar Baby released in t...,"December 15, 2004"
5,Crash,When was Crash released on DVD?,"September 6, 2005"
6,The Departed,How much did The Departed gross on opening wee...,$26.9 million
7,No Country for Old Men,The film No Country for Old Men is based on th...,Cormac McCarthy
8,Slumdog Millionaire,Slumdog Millionaire is based on what novel?,Q & A
9,The Hurt Locker,Where was The Hurt Locker filmed?,Jordan


In [13]:
def context(question):
    results = collection.query(
    query_texts = [question],
    n_results = 5
    )
    return '\n'.join(results['documents'][0])

In [14]:
questions['context']=  questions['question'].apply(context)

In [15]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

C:\Users\micha\anaconda3\envs\chromadb_ds\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
embeddings = questions['context'].apply(encoder.encode)

In [17]:
decoder = pipeline('text-generation', model='distilgpt2')

In [18]:
qa = pipeline(task="question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [19]:
def answer_generation(row):
    question = row['question']
    context = row['context']  
    try:
        answer = qa(question=question, context=context)
        return answer['answer']
    except ValueError as e:
        print(f"Error generating answer for question '{question}': {e}")
        return "Error: Unable to generate answer"


In [20]:
with tqdm(total=len(questions)) as pbar:
    def update_pbar(*args):
        pbar.update()
    
    def apply_with_progress(row):
        answer = answer_generation(row)
        update_pbar()
        return answer
    
    questions['answer_generated'] = questions.apply(apply_with_progress, axis=1)

  0%|          | 0/24 [00:00<?, ?it/s]

In [21]:
questions[['question','answer','answer_generated']]

,question,answer,answer_generated
0,What was the worldwide gross of Gladiator?,$465.4 million,250 1 million
1,Who was consulted on the mathematical equation...,Dave Bayer,Barry Jenkins
2,Where did principal photography take place for...,"Toronto, Ontario, Canada.",Boston Massachusetts 26
3,Who composed the score for the film The Lord o...,Howard Shore,Peter Jackson
4,What day was Million Dollar Baby released in t...,"December 15, 2004",December 15 2004
5,When was Crash released on DVD?,"September 6, 2005",May 6 2005
6,How much did The Departed gross on opening wee...,$26.9 million,26 9 million
7,The film No Country for Old Men is based on th...,Cormac McCarthy,Cormac McCarthy
8,Slumdog Millionaire is based on what novel?,Q & A,Nomadland Surviving America in the Twenty Firs...
9,Where was The Hurt Locker filmed?,Jordan,Rob Marshall Chicago


In [22]:
import numpy as np
np.sum(questions['answer']== questions['answer_generated'])

3